## 🤔 Device check

In [1]:
!nvidia-smi

Sat Dec 16 00:21:51 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:3B:00.0 Off |                    0 |
| N/A   37C    P0              44W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 🛠️ Packages

In [2]:
import warnings
warnings.filterwarnings(action="ignore")

In [3]:

import lightning.pytorch as pl
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import RichProgressBar, TQDMProgressBar, ModelCheckpoint
from lightning.pytorch.callbacks.progress.rich_progress import RichProgressBarTheme
from lightning.pytorch.loggers import TensorBoardLogger, WandbLogger

import math

import logging
logging.basicConfig(level="INFO")

import math
from matplotlib import pyplot
%matplotlib inline

import numpy as np

import os

import pandas as pd
from pprint import pprint

import random

import sys
sys.path.append("../src")

import time
import timm
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

import wandb

In [4]:
%load_ext autoreload
%autoreload 2

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["WANDB__SERVICE_WAIT"] = "300"

In [5]:
import config
from dataloader import BEDataset, BEDataModule
from transformer import make_attn_mask

from rt1 import RT1CRAM
from utils.model_utils import plot_attention, fetch_sample_from_batch
import utils.data_utils as data_utils

## 📊 Data Module

In [6]:
dm = BEDataModule()
dm.setup()

INFO:root:Training on 3848 samples.
INFO:root:Validating on 660 samples.
INFO:root:Testing on 250 samples.


Total # examples: 4758


In [7]:
%%time
batch = next(iter(dm.train_dataloader()))
print(batch.keys())
batch["in_state"].shape

dict_keys(['sample_id', 'in_state', 'action_desc', 'motor_cmd'])
CPU times: user 533 ms, sys: 552 ms, total: 1.09 s
Wall time: 10.9 s


torch.Size([32, 3, 288, 288])

In [8]:
inp = fetch_sample_from_batch(
    batch, 
    batch_size=batch["in_state"].shape[0],
    random=True
)

In [9]:
inp.keys()

dict_keys(['sample_id', 'in_state', 'raw_action_desc', 'ids', 'mask', 'token_type_ids', 'raw_motor_cmd', 'decoder_inp_ids', 'labels'])

In [10]:
dm.train_ds.tokenizer.decode(batch["action_desc"]["ids"][0])

'[CLS] put the milk to the left of bowl [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [11]:
batch["action_desc"]["raw"][0]

'put the milk to the left of bowl'

## 🤖 RT1-CRAM

In [12]:
rt1 = RT1CRAM(
    cnn_bacnbone=config.SELECTED_CNN_BACKBONE, 
    num_res_blocks=config.NUM_RES_BLOCKS,
    freeze_cnn_backbone=config.FREEZE_CNN
).cuda()
# print(rt1)

summary(model=rt1)

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b3.ra2_in1k)
INFO:timm.models._hub:[timm/efficientnet_b3.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


Layer (type:depth-idx)                                            Param #
RT1CRAM                                                           --
├─RT1Encoder: 1-1                                                 --
│    └─TextEncoder: 2-1                                           --
│    │    └─BertModel: 3-1                                        (28,763,648)
│    │    └─Dropout: 3-2                                          --
│    └─FiLMEncoder: 2-2                                           --
│    │    └─ImageFeatureExtractor: 3-3                            10,300,456
│    │    └─ModuleList: 3-4                                       4,227,072
│    └─TokenLearnerV11: 2-3                                       --
│    │    └─Sequential: 3-5                                       134,408
├─RT1Decoder: 1-2                                                 --
│    └─TransformerDecoder: 2-4                                    --
│    │    └─EmbeddingLayer: 3-6                                   53

## 🏋️‍ Training

In [13]:
loss_fn = nn.CrossEntropyLoss(
    ignore_index=config.TGT_PAD_TOK_ID, 
    label_smoothing=config.LABEL_SMOOTHING
)

opt = getattr(torch.optim, config.OPTIMIZER)(
    params=[p for p in rt1.parameters() if p.requires_grad], 
    lr=config.LR,
    weight_decay=config.WEIGHT_DECAY
)

scheduler = getattr(torch.optim.lr_scheduler, config.LR_SCHEDULER["type"])(**config.LR_SCHEDULER["params"], optimizer=opt)

opt

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 1.0
    lr: 1e-05
    maximize: False
    weight_decay: 2e-06
)

## Training step

In [14]:
def training_step(model, batch, loss_fn):

    input_ids=batch["action_desc"]["ids"].to(config.DEVICE)
    attn_mask=batch["action_desc"]["mask"].to(config.DEVICE)
    token_type_ids=batch["action_desc"]["token_type_ids"].to(config.DEVICE)
    imgs=batch["in_state"].to(config.DEVICE)
    decoder_inp=batch["motor_cmd"]["decoder_inp_ids"].to(config.DEVICE)
    
    # forward
    logits, self_attn_ws, cross_attn_ws = model(
        input_ids=input_ids, 
        attn_mask=attn_mask, 
        token_type_ids=token_type_ids, 
        imgs=imgs,
        decoder_inp=decoder_inp
    )

    # loss computation
    labels = batch["motor_cmd"]["labels"].to(config.DEVICE)
    loss = loss_fn(logits.view(-1, logits.shape[2]), labels.view(-1))
        
    return loss, logits, self_attn_ws, cross_attn_ws

In [15]:
loss, logits, self_attn_ws, cross_attn_ws = training_step(model=rt1, batch=batch, loss_fn=loss_fn)

In [16]:
logits.shape

torch.Size([32, 16, 52])

In [17]:
# predictions
preds = logits.softmax(dim=-1).argmax(dim=-1)
preds.shape, preds

(torch.Size([32, 16]),
 tensor([[19, 44, 42, 44, 42, 27,  9, 40, 49, 34, 44, 43, 43, 19,  6, 43],
         [44, 32, 14, 11, 40, 27, 40, 44, 46, 43, 26, 19, 44, 36, 14, 14],
         [32, 40, 32, 44, 27, 23, 46, 40,  8, 32, 19, 44,  6, 44,  8,  8],
         [32,  6, 32, 26, 46, 48, 48, 44, 29, 34, 34, 44, 19,  8,  6, 17],
         [14, 51, 30, 19,  8, 36, 14, 50, 40,  8, 34, 43, 33, 32, 32,  6],
         [32, 27,  0, 47,  5, 28, 38, 27, 49, 22, 17, 44, 19, 19, 50, 19],
         [34, 48, 44,  5, 48, 50, 34, 14, 44, 48, 22, 33,  6, 49, 44, 32],
         [32, 14, 33, 50, 25,  1, 28, 43, 17, 17, 43,  6,  6,  8,  6, 19],
         [14, 34, 44, 29, 17, 28, 42, 44, 34, 18, 43, 14, 43, 32, 32, 14],
         [14, 14, 19, 32, 44, 28, 48, 25, 32, 50, 44, 32, 51, 19, 19, 14],
         [32, 34, 19, 34, 50, 19, 11, 34, 34, 44, 44, 32, 11,  6, 17, 43],
         [44, 44, 32, 34, 44, 19, 23, 46, 34, 50,  8,  8,  6,  6, 32, 35],
         [32, 19, 45, 19,  5, 48, 38,  8,  5, 41, 19, 12, 32, 19, 32, 19],
  

In [18]:

# decode predictions
preds = rt1.decode_predictions(
    predicted_ids=preds
)

preds[0]

":MUG #'*forward-transformation* #'*on-transformation* #'*forward-transformation* #'*on-transformation* POSE-4 :GLASSES POSE-2 POSE-13 POSE-5 #'*forward-transformation* #'*leftward-transformation* #'*leftward-transformation* :MUG :SHOE #'*leftward-transformation*"

In [19]:
lonpkv;/ad

SyntaxError: invalid syntax (980487135.py, line 1)

## 🛟 Greedy decoding

In [20]:
def greedy_decoding(
    model:pl.LightningModule, 
    batch_inp:dict, 
    max_len:int=config.MAX_OUT_SEQ_LEN, 
    debug:bool=False
):
    if model.device.type == "cpu":
        model.to(config.DEVICE)
    model.eval()
    
    sos_token = config.TARGETS_MAPPING["[SOS]"]
    eos_token = config.TARGETS_MAPPING["[EOS]"]
    
    input_ids=batch_inp["ids"].to(config.DEVICE)
    attn_mask=batch_inp["mask"].to(config.DEVICE)
    token_type_ids=batch_inp["token_type_ids"].to(config.DEVICE)
    imgs=batch_inp["in_state"].to(config.DEVICE)

    _, learned_tokens = model._encode(
        input_ids=input_ids, 
        attn_mask=attn_mask, 
        token_type_ids=token_type_ids, 
        imgs=imgs    
    )
    
    decoder_inp = torch.empty(1, 1, dtype=torch.long, device=input_ids.device).fill_(sos_token)

    for t in range(config.MAX_OUT_SEQ_LEN):
        mask = make_attn_mask(dim=decoder_inp.shape[1])

        with torch.no_grad():
            logits, self_attn_ws, cross_attn_ws = rt1._decode(
            decoder_inp=decoder_inp, 
            encoder_out=learned_tokens,
            attn_mask=mask,
            return_actions=False
        )

        # perform greedy decoding
        probs = rt1.decoder.action_generator(logits[:, -1])

        _, next_tok = torch.max(probs, dim=-1)
        # update decoder input
        decoder_inp = torch.cat((decoder_inp, next_tok.unsqueeze(1)), dim=1)
            
    return decoder_inp[:, 1:].cpu().detach(), logits, self_attn_ws.cpu().detach(), cross_attn_ws.cpu().detach()

## Validation Step

In [21]:
def validation_step(batch, model, loss_fn, debug:bool=False):
    
    inp = fetch_sample_from_batch(
        batch, 
        batch_size=batch["in_state"].shape[0],
        random=True
    )
    
    pred_ids, logits, self_attn_ws, cross_attn_ws = greedy_decoding(
        model=model, 
        batch_inp=inp, 
        debug=debug
    )
    
    labels = inp["labels"].to(config.DEVICE)
    
    preds = model.decode_predictions(
            predicted_ids=pred_ids
    )[0]

    label = model.decode_predictions(
        predicted_ids=labels
    )[0]  
    
    # compute metrics
    val_loss = loss_fn(logits.view(-1, logits.shape[2]), labels.view(-1)).item()  # loss
    cer = model.cer_fn(preds, label).item() # Character Error Rate
    wer = model.wer_fn(preds, label).item() # Word Error Rate
    
    output = {
        "val_loss"              : val_loss,
        "CER"                   : cer,
        "WER"                   : wer,
        "label"                 : label,
        "pred_ids"              : pred_ids,
        "pred_tokens"           : preds,
        "self_attn_ws"          : self_attn_ws, 
        "cross_attn_ws"         : cross_attn_ws
    }
    
    return output

In [22]:
%%time

out = validation_step(model=rt1, batch=batch, loss_fn=loss_fn)

out

CPU times: user 235 ms, sys: 8.05 ms, total: 243 ms
Wall time: 779 ms


{'val_loss': 7.094069480895996,
 'CER': 0.8500000238418579,
 'WER': 1.0,
 'label': ":BOWL GREEN POSE-10 :BOWL #'*rightward-transformation* :BOWL",
 'pred_ids': tensor([[32, 11,  5, 32,  2, 32,  2, 32,  2, 32,  2, 32,  2, 32,  2, 32]]),
 'pred_tokens': 'POSE-11 :CEREAL :SPOON POSE-11',
 'self_attn_ws': tensor([[[[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [2.2027e-02, 9.7797e-01, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [3.7161e-01, 8.8347e-02, 5.4005e-01,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           ...,
           [3.8384e-02, 4.2258e-02, 3.1755e-04,  ..., 1.6146e-02,
            0.0000e+00, 0.0000e+00],
           [3.1579e-03, 1.3016e-01, 3.4435e-03,  ..., 3.1899e-02,
            9.4373e-02, 0.0000e+00],
           [1.7627e-02, 4.2222e-02, 5.8494e-04,  ..., 1.0933e-02,
            1.6845e-02, 9.8074e-03]],
 
          [[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e

In [24]:
rt1.decode_predictions(predicted_ids=out["pred_ids"])

['POSE-11 :CEREAL :SPOON POSE-11']

## 🧑🏾‍🍳 Prepare Experiment

In [25]:
def run_experiment(model, dm, opt, loss_fn, scheduler):
    
    loss_epoch = np.inf
    val_loss = np.inf
    best_val_loss = np.inf
    
    cer_ = np.inf
    wer_ = np.inf
    
    for e in range(config.EPOCHS):        
        running_loss = 0.
        num_steps = len(dm.train_dataloader())
        
        pbar = tqdm(
            range(num_steps),
            position=0,
            leave=True,
            dynamic_ncols=True,
            total = num_steps
        )
        
        # training
        model.train()
        for step, batch in enumerate(dm.train_dataloader()):            
            pct = 100. * step / num_steps
            pbar.set_description(
                f"Epoch {e+1}/{config.EPOCHS} - (Train {pct:.1f}%)"
            )
            pbar.update()
            
            opt.zero_grad()

            # training step
            loss, logits, self_attn_ws, cross_attn_ws = training_step(
                model=model, 
                batch=batch, 
                loss_fn=loss_fn
            )
            
            # plot attention weights
            plot_attention(
                self_attn_ws, 
                show=False, 
                pre_fix="train_selfattn", 
                folder="train",
                epoch=e,
                wandb_logging=True
            )

            plot_attention(
                cross_attn_ws,
                kind="cross", 
                pre_fix="train_crossattn", 
                show=False, 
                folder="train",
                epoch=e,
                wandb_logging=True
            )   
            
            running_loss += loss.item()         
            
            # logging
            if step % 10 == 0:
                pbar.set_postfix(
                    train_loss_step="{:.04f}".format(running_loss/(step+1)),
                    train_loss="{:.04f}".format(loss_epoch),
                    CER="{:.04f}".format(cer_),
                    WER="{:.04f}".format(wer_),
                    val_loss="{:.04f}".format(val_loss),
                )
                pbar.update()

            # backward
            loss.backward()
            
            # Adjust learning weights
            opt.step()
            
        loss_epoch = running_loss / len(dm.train_dataloader())   
        final_lr_epoch = float(opt.param_groups[0]['lr'])
        
        # predictions
        preds = logits.softmax(dim=-1).argmax(dim=-1)

        # decode predictions
        preds = model.decode_predictions(
            predicted_ids=preds
        )

        labels = model.decode_predictions(
            predicted_ids=batch["motor_cmd"]["labels"]
        )         
            
        # log decoded sentenses
        with open(config.LOGGING_FILE, "a") as f:            
            f.write(f"Epoch #{e+1}\n")
            f.write(f"[Train] \n")
            
            pred = preds[0]
            label = labels[0]
            
            cer_ = model.cer_fn(pred, label).item()
            wer_ = model.wer_fn(pred, label).item()
            f.write(f"Predicted \t: {pred}\n")
            f.write(f"Actual \t\t: {label}\n")
                
        # validation
        out = validation_step(model=rt1, batch=batch, loss_fn=loss_fn)
        val_loss = out["val_loss"]
        
        # start scheduling lr after epoch X
        # X set to 30 to start us of
        if e >=30:
            scheduler.step(val_loss)
       
        # plot attention weights
        plot_attention(
            out["self_attn_ws"], 
            show=False, 
            pre_fix="val_selfattn", 
            folder="val",
            epoch=e,
            wandb_logging=True
        )

        plot_attention(
            out["cross_attn_ws"],
            kind="cross", 
            pre_fix="val_crossattn", 
            show=False, 
            folder="val",
            epoch=e,
            wandb_logging=True
        )   
        
        # update best score
        if val_loss < best_val_loss:
            # save checkpoint
            path = os.path.join(config.MODEL_PATH, "be_model.bin")
            torch.save({
                'model_state_dict'      :model.state_dict(),
                'optimizer_state_dict'  :opt.state_dict(),
                'val_loss'              : val_loss, 
                'epoch'                 : e
                }, path)
            
            # update best score
            best_val_loss = val_loss        
        
        pbar.set_postfix(
            train_loss_step="{:.04f}".format(running_loss/(step+1)),
            train_loss="{:.04f}".format(loss_epoch),
            # CER="{:.04f}".format(cer_),
            # WER="{:.04f}".format(wer_),
            val_Loss="{:.04f}".format(val_loss),
            val_CER="{:.04f}".format(out["CER"]),
            val_WER="{:.04f}".format(out["WER"]),
            lr_epoch="{:.1e}".format(final_lr_epoch),
        )  
        pbar.update()
        
        logs_dict = {
            "epoch" :e,
            "train_loss":loss_epoch,
            "val_loss":val_loss,
            "val_CER":out["CER"],
            "valWER":out["WER"],
            "lr":final_lr_epoch
        }
        wandb.log(logs_dict)
        
        # log decoded sentenses
        with open(config.LOGGING_FILE, "a") as f:                        
            pred = out["pred_tokens"]
            label = out["label"]
            
            f.write(f"[Val] \n")            
            f.write(f"Predicted \t: {pred}\n")
            f.write(f"Actual \t\t: {label}\n") 
            f.write(f"Curr val loss \t\t: {val_loss:.5f}\n") 
            f.write(f"Best loss: \t\t: {best_val_loss:.5f}\n\n") 
            
        pbar.close()
        torch.cuda.empty_cache()
        
    return model

## 🚀 Run Experiment

In [26]:
# Set seed

random.seed(config.SEED)
np.random.seed(config.SEED)
torch.manual_seed(config.SEED)

if torch.cuda.is_available(): 
    torch.cuda.manual_seed(config.SEED)
    torch.cuda.manual_seed_all(config.SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
##### init experiment
run = wandb.init(
    project='SMF-Be', 
    group="RT1-CRAM", 
    name="be_model", 
    reinit=True
)

with open(config.LOGGING_FILE, "a") as f:   
    f.write("*** New experiment ***\n")
    
    
trained_model = run_experiment(
    model=rt1, 
    dm=dm, 
    opt=opt, 
    loss_fn=loss_fn,
    scheduler=scheduler
)


wandb.finish()


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dric225 (jepsam-s23). Use `wandb login --relogin` to force relogin


  0%|                                                                                                         …

  0%|                                                                                                         …

  0%|                                                                                                         …

  0%|                                                                                                         …

In [ ]:
# wandb.finish()

## 👨🏿‍🔬 Test / Inference

In [ ]:
# input_ids=batch["action_desc"]["ids"].cuda()
# attn_mask=batch["action_desc"]["mask"].cuda()
# token_type_ids=batch["action_desc"]["token_type_ids"].cuda()
# imgs=batch["in_state"].cuda()
# decoder_inp=batch["motor_cmd"]["decoder_inp_ids"].cuda()
# src_mask=(batch["source_mask"].cuda(), batch["source_mask_tokens"].cuda())
# target_mask=batch["target_mask"].cuda()
# labels = batch["motor_cmd"]["labels"].cuda()